# ETL Part 1 = Extract the data from the database

In [1]:
import psycopg2 # Library used to connect python to Postgres database, Postgres database adapter for Python
import pandas as pd

In [2]:
import config as cfg

con = psycopg2.connect(database = 'mid_term_project', user = 'lhl_student', password = 'lhl_student',
                       host = 'mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com', port = '5432')  # This should work if you left everything as default

print("Database opened successfully")

cur = con.cursor()


Database opened successfully


In [3]:
cur

<cursor object at 0x0000022B35109F28; closed: 0>

In [4]:
def execute_query(query_string, return_pandas=True):
    if return_pandas:
        response = pd.read_sql_query(query_string, con)
    else:
        cur.execute(query_string)
        response = cur.fetchall()
        
    print("Operation done successfully")
#     con.close()
    return response


### flights

I will only be importing data for Jan 2019, the first week of data has c. 150,000 instances. Reason beign is that 
we are being asked to predict delays on flights from first 7 days of 2020 (1st of January - 7th of January).

In [5]:
# Let's try out our function to make sure it does
# the same as what we have above
query = """
SELECT * 
FROM flights
WHERE fl_date BETWEEN '2019-01-01' AND '2019-01-07';
"""
execute_query(query)

# query = """
# SELECT * 
# FROM flights
# LIMIT 5;
# """

Operation done successfully


,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-01-01,AA,AA_CODESHARE,AA,3069,OO,N727SK,3069,14107,PHX,...,110.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1,2019-01-03,AA,AA_CODESHARE,AA,3607,MQ,N820AE,3607,15074,SWO,...,225.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
2,2019-01-01,AA,AA_CODESHARE,AA,5606,OH,N575NN,5606,13577,MYR,...,157.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,10529,BDL,...,313.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
4,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,11278,DCA,...,313.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146480,2019-01-07,AA,AA_CODESHARE,AA,4517,YX,N113HQ,4517,10693,BNA,...,562.0,0.0,0.0,16.0,0.0,0.0,None,NaN,NaN,None
146481,2019-01-07,AA,AA_CODESHARE,AA,4517,YX,N113HQ,4517,11278,DCA,...,562.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
146482,2019-01-07,AA,AA_CODESHARE,AA,4519,YX,N118HQ,4519,15016,STL,...,814.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
146483,2019-01-07,AA,AA_CODESHARE,AA,4520,YX,N126HQ,4520,14100,PHL,...,588.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None


In [6]:
flights_df = execute_query(query).copy() # I set the database to the function call to which the query arument was passed. The return value which is a df will be set to the varible df name here. I put .copy() at the end as the function will be called more than once so that the variable df data wont be changed for any new subsequent queries.


Operation done successfully


In [7]:
flights_df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-01-01,AA,AA_CODESHARE,AA,3069,OO,N727SK,3069,14107,PHX,...,110.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1,2019-01-01,AA,AA_CODESHARE,AA,5606,OH,N575NN,5606,13577,MYR,...,157.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
2,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,10529,BDL,...,313.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3,2019-01-01,AA,AA_CODESHARE,AA,5607,OH,N254PS,5607,11278,DCA,...,313.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
4,2019-01-01,AA,AA_CODESHARE,AA,5608,OH,N710PS,5608,11278,DCA,...,396.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None


In [8]:
# Saving the flights fact table as a csv:
flights_df.to_csv('data/flights_df.csv', index = False) # I set index = False as an extra column which was equal to the index was beign created.

### fuel_comsumption

In [9]:
query = """
SELECT * 
FROM fuel_comsumption 
WHERE 
month = 1 AND
year = 2019;
"""

# query = """
# SELECT * 
# FROM fuel_comsumption 
# LIMIT 5;
# """
execute_query(query)

Operation done successfully


,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,21161.0,09Q,09Q,"Swift Air, LLC d/b/a Eastern Air Lines d/b/a E...",2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1831212.0,1618745.0,3449957.0,2019
1,1,21352.0,0WQ,0WQ,Avjet Corporation,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
2,1,21492.0,1BQ,1BQ,"Eastern Airlines f/k/a Dynamic Airways, LLC",1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,102795.0,74193.0,176988.0,2019
3,1,21652.0,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
4,1,21712.0,2HQ,2HQ,Elite Airways LLC,1,66815.0,0.0,0.0,0.0,...,175499.0,0.0,0.0,0.0,0.0,175499.0,1518919.0,0.0,1518919.0,2019
5,1,21974.0,3EQ,3EQ,"Scott Aviation, LLC d/b/a Silver Air",1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
6,1,20408.0,5V,5V,Tatonduk Outfitters Limited d/b/a Everts Air A...,1,287034.0,0.0,0.0,0.0,...,746126.0,0.0,0.0,0.0,0.0,746126.0,792440.0,0.0,792440.0,2019
7,1,19917.0,5X,5X,United Parcel Service,3,41673000.0,12332000.0,16238000.0,2967000.0,...,75938000.0,22394000.0,29385000.0,5845000.0,57624000.0,133562000.0,75938000.0,57624000.0,133562000.0,2019
8,1,20007.0,5Y,5Y,Atlas Air Inc.,3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,30473337.0,61310144.0,91783481.0,2019
9,1,19874.0,8C,8C,Air Transport International,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,43613.0,1910733.0,1954346.0,2019


In [10]:
fuel_comsumption_df = execute_query(query).copy()

Operation done successfully


In [11]:
fuel_comsumption_df.head()

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,...,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,21161.0,09Q,09Q,"Swift Air, LLC d/b/a Eastern Air Lines d/b/a E...",2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1831212.0,1618745.0,3449957.0,2019
1,1,21352.0,0WQ,0WQ,Avjet Corporation,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
2,1,21492.0,1BQ,1BQ,"Eastern Airlines f/k/a Dynamic Airways, LLC",1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,102795.0,74193.0,176988.0,2019
3,1,21652.0,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
4,1,21712.0,2HQ,2HQ,Elite Airways LLC,1,66815.0,0.0,0.0,0.0,...,175499.0,0.0,0.0,0.0,0.0,175499.0,1518919.0,0.0,1518919.0,2019


In [12]:
# Saving the fuel_comsumption dimension table as a csv:
fuel_comsumption_df.to_csv('data/fuel_comsumption_df.csv', index = False)

### passengers

In [13]:
query = """
SELECT * 
FROM passengers 
WHERE 
month = 1 AND
year = 2019;
"""

# query = """
# SELECT * 
# FROM passengers 
# LIMIT 5;
# """
execute_query(query)

Operation done successfully


,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,0.0,1.0,42500.0,173.0,0.0,0.0,0.0,389.0,70.0,55.0,...,US,United States,6,614,1,2019,1,1,L,DU
1,0.0,4.0,124800.0,624.0,488.0,0.0,0.0,1396.0,942.0,768.0,...,MX,Mexico,6,698,1,2019,1,3,L,IU
2,0.0,4.0,124800.0,624.0,543.0,0.0,0.0,1667.0,1050.0,894.0,...,DO,Dominican Republic,6,698,1,2019,1,4,L,IU
3,0.0,1.0,37200.0,186.0,186.0,0.0,0.0,1716.0,270.0,240.0,...,US,United States,6,694,1,2019,1,4,L,IU
4,0.0,3.0,93600.0,468.0,388.0,0.0,0.0,1667.0,750.0,660.0,...,US,United States,6,698,1,2019,1,4,L,IU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40474,905.0,905.0,1086000.0,4977.0,3324.0,6544.0,0.0,11.0,11682.0,8967.0,...,US,United States,0,79,1,2019,1,1,F,DU
40475,923.0,923.0,1107600.0,5074.0,3350.0,14291.0,0.0,11.0,12987.0,10218.0,...,US,United States,0,79,1,2019,1,1,F,DU
40476,2170.0,46.0,74440.0,276.0,0.0,0.0,46.0,59.0,1860.0,0.0,...,US,United States,3,340,1,2019,1,1,F,DU
40477,400.0,399.0,12603800.0,51072.0,41198.0,101367.0,0.0,216.0,21656.0,14945.0,...,US,United States,6,608,1,2019,1,1,F,DU


In [14]:
passengers_df = execute_query(query).copy()

Operation done successfully


In [15]:
passengers_df.head()

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,0.0,4.0,124800.0,624.0,488.0,0.0,0.0,1396.0,942.0,768.0,...,MX,Mexico,6,698,1,2019,1,3,L,IU
1,0.0,1.0,18800.0,70.0,0.0,0.0,0.0,83.0,43.0,18.0,...,US,United States,6,631,1,2019,1,1,F,DU
2,0.0,1.0,22750.0,76.0,17.0,0.0,0.0,308.0,75.0,55.0,...,US,United States,6,638,1,2019,1,1,F,DU
3,0.0,1.0,18800.0,70.0,33.0,0.0,0.0,166.0,73.0,37.0,...,US,United States,6,631,1,2019,1,1,F,DU
4,0.0,3.0,56400.0,210.0,108.0,0.0,0.0,152.0,159.0,100.0,...,US,United States,6,631,1,2019,1,1,F,DU


In [16]:
# Saving the passengers_df dimension table as a csv:
passengers_df.to_csv('data/passengers_df.csv', index = False)

### flights_test

In [17]:
query = """
SELECT * 
FROM flights_test 
LIMIT 5;
"""
execute_query(query)

Operation done successfully


,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


In [18]:
flights_test_df = execute_query(query).copy()

Operation done successfully


In [19]:
flights_test_df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


In [20]:
# Saving the flights_test_df testing table as a csv:
flights_test_df.to_csv('data/flights_test_df.csv', index = False)

#### Checking the shape to ensure that each df is indeed different as the same function is used to get each of them, where the return is 'response'

In [21]:
flights_df.head(3).shape

(3, 42)

In [22]:
fuel_comsumption_df.head(3).shape

(3, 25)

In [23]:
passengers_df.head(3).shape

(3, 38)

In [24]:
flights_test_df.head(3).shape

(3, 20)

# end